In [61]:
# Please refer "Codes/B2B_FRISM_Sep.ipynb" which ran properly if it has any issue 

In [62]:
### Simulation flow
# 1. Assign yearly to daily
# 2. Assign shipments to carriers
# 3. 

In [63]:
import pandas as pd
import numpy as np
import geopandas as gpd
import seaborn as sns
import random

In [96]:
### main output structure
fdir_main_output= "../Sim_outputs/Shipment2Fleet/"
fnm_payload="B2B_payload_1209_21.csv"
fnm_carrier="B2B_carrier_1209_21.csv"
fnm_vtype="vehicle_types.csv"

In [65]:
### Read inputs
### firm2firm annual shipments
fdir_synth_firm='../Sim_inputs/Synth_firm_results/'

In [66]:
B2BF_C1 =pd.read_csv(fdir_synth_firm+'truck_only_sctg1_OD.csv', header=0, sep=',')
B2BF_C2=pd.read_csv(fdir_synth_firm+'truck_only_sctg2_OD.csv', header=0, sep=',')
B2BF_C3=pd.read_csv(fdir_synth_firm+'truck_only_sctg3_OD.csv', header=0, sep=',')
B2BF_C4= pd.read_csv(fdir_synth_firm+'truck_only_sctg4_OD.csv', header=0, sep=',')
B2BF_C5= pd.read_csv(fdir_synth_firm+'truck_only_sctg5_OD.csv', header=0, sep=',')

/Users/kjeong/anaconda3/envs/python38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (2,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/kjeong/anaconda3/envs/python38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [67]:
#### Dictionary of Synth firm outputs
# BuyerID: the ID of the buyer	
# BuyerZone: mesozone ID of the buyer
# BuyerNAICS: the 6-digit NAICS code of the buyer
# SellerID: the ID of the seller
# SellerZone: mesozone ID of the seller
# SellerNAICS: the 6-digit NAICS code of the seller
# TruckLoad: the shipment load in tons (I converted the unit for mode choice model).
# SCTG_Group: the commodity group of the shipment (1 - bulk, 2 - fuel_fert, 3 - interm_food, 4 - mfr_good, 5 - others)
# Distance: the great circle distance in miles between the origin and destination assigned through post-process.
# Mode_choice: private truck or for-hire truck
# Probability: the probability of selected mode from MNL model


In [68]:
### other inputs

In [87]:
fdir_firms='../Sim_inputs/Synth_firm_pop/'

firms= pd.read_csv(fdir_firms+'synthfirms_all_Sep.csv', header=0, sep=',')
warehouses= pd.read_csv(fdir_firms+'warehouses_all_Sep.csv', header=0, sep=',')

/Users/kjeong/anaconda3/envs/python38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [70]:
fdir_geo='../Sim_inputs/Geo_data/'

dist_df=pd.read_csv(fdir_geo+'od_distance.csv', header=0, sep=',')
CBGzone_df = gpd.read_file(fdir_geo+'freight_centroids.geojson')

In [71]:
fdir_truck='../Model_carrier_op/INRIX_processing/'

df_dpt_dist=pd.read_csv(fdir_truck+'depature_dist_by_cbg_HD.csv', header=0, sep=',')

In [72]:
### Process Shipment data

In [73]:
# Covert tons to pounds for TruckLoad (the shipment load in tons)
B2BF_C1['TruckLoad']=B2BF_C1['TruckLoad']*2000
B2BF_C2['TruckLoad']=B2BF_C2['TruckLoad']*2000
B2BF_C3['TruckLoad']=B2BF_C3['TruckLoad']*2000
B2BF_C4['TruckLoad']=B2BF_C4['TruckLoad']*2000
B2BF_C5['TruckLoad']=B2BF_C5['TruckLoad']*2000

In [74]:
# initialize daily shipment and day selection 
B2BF_C1["D_truckload"]=0
B2BF_C1["D_selection"]=0
B2BF_C2["D_truckload"]=0
B2BF_C2["D_selection"]=0
B2BF_C3["D_truckload"]=0
B2BF_C3["D_selection"]=0
B2BF_C4["D_truckload"]=0
B2BF_C4["D_selection"]=0
B2BF_C5["D_truckload"]=0
B2BF_C5["D_selection"]=0

In [75]:
# need to update this for commodity specific for daily
def d_load(TruckLoad):
    if TruckLoad <= 265*1:
        return float(TruckLoad)
    elif TruckLoad <= 265*10 and TruckLoad > 265*1:
        return float(TruckLoad/4)
    elif TruckLoad <= 265*100 and TruckLoad > 265*10:
        return float(TruckLoad/12)          
    elif TruckLoad > 265*100:
        return float(TruckLoad/265)   
    
def d_select(TruckLoad):
    if TruckLoad <= 265*1:
         if np.random.uniform(0,256) <=1:
            return 1
         else: return 0   
    elif TruckLoad <= 265*10 and TruckLoad > 265*1:
        if np.random.uniform(0,int(256/4)) <=1:
            return 1
        else: return 0  
    elif TruckLoad <= 265*100 and TruckLoad > 265*10:
         if np.random.uniform(0,int(256/12)) <=1:
            return 1
         else: return 0  
    elif TruckLoad > 265*100:
         if np.random.uniform(0,int(256/48)) <=1:
            return 1
         else: return 0 
    else: return 0      

In [76]:
B2BF_C1.D_truckload=B2BF_C1['TruckLoad'].apply(d_load)
B2BF_C1.D_selection=B2BF_C1['TruckLoad'].apply(d_select)

B2BF_C2.D_truckload=B2BF_C2['TruckLoad'].apply(d_load)
B2BF_C2.D_selection=B2BF_C2['TruckLoad'].apply(d_select)

B2BF_C3.D_truckload=B2BF_C3['TruckLoad'].apply(d_load)
B2BF_C3.D_selection=B2BF_C3['TruckLoad'].apply(d_select)

B2BF_C4.D_truckload=B2BF_C4['TruckLoad'].apply(d_load)
B2BF_C4.D_selection=B2BF_C4['TruckLoad'].apply(d_select)

B2BF_C5.D_truckload=B2BF_C5['TruckLoad'].apply(d_load)
B2BF_C5.D_selection=B2BF_C5['TruckLoad'].apply(d_select)

In [77]:
B2BF_C1_D=B2BF_C1.query('D_selection ==1')
B2BF_C2_D=B2BF_C2.query('D_selection ==1')
B2BF_C3_D=B2BF_C3.query('D_selection ==1')
B2BF_C4_D=B2BF_C4.query('D_selection ==1')
B2BF_C5_D=B2BF_C5.query('D_selection ==1')
B2BF_T_D=pd.concat([B2BF_C1_D,B2BF_C2_D,B2BF_C3_D,B2BF_C4_D,B2BF_C5_D ])

In [78]:
## temporary saving
B2BF_T_D.to_csv ('../Sim_outputs/temp_save/B2B_daily.csv', index = False, header=True)

## read tempoary data
#B2BF_T_D = pd.read_csv('../Sim_outputs/temp_save/B2B_daily.csv', header=0, sep=',')

In [79]:
## Deal with the daily shipment seperately
## For-hire Truck: FH_B2B
## Private Truck: PV_B2B
FH_B2B= B2BF_T_D[B2BF_T_D['mode_choice']=="For-hire Truck"]
PV_B2B= B2BF_T_D[B2BF_T_D['mode_choice']=="Private Truck"]

In [80]:
### Alameda county sample: This is for temporary run; need to just delete it without any modification later 
### (FH_Seller (full version) is replaced with the same file name, so it's okay to remove for the entire run)
#####  Update requried: later extract it and modulize? 
# To extact sample in county == 1(Alameda county) in SF Bay area

sel_zone= pd.read_csv(fdir_geo+'selected zone.csv')
sel_zone=sel_zone.rename({'blkgrpid':'GEOID'}, axis=1)
sel_zone=sel_zone.merge(CBGzone_df, on="GEOID", how='left')

FH_B2B=FH_B2B[FH_B2B['SellerZone'].isin(sel_zone['MESOZONE'])].reset_index()
PV_B2B=PV_B2B[PV_B2B['SellerZone'].isin(sel_zone['MESOZONE'])].reset_index()

In [81]:
## Assign shipment can be hanlded in Private truck to For-hire truck 
## Rule base: for capacity
PV_B2B =PV_B2B.sort_values(by=['SellerID', 'Distance']).reset_index()

sel_ID=0
for i in range (0,PV_B2B.shape[0]):
    if (sel_ID != PV_B2B.SellerID[i]):
        fleet_capacity = firms[firms.BusID ==PV_B2B.SellerID[i]]['md_veh'].values[0]*14000 + firms[firms.BusID ==PV_B2B.SellerID[i]]['hd_veh'].values[0]*40000
        sel_ID=PV_B2B.SellerID[i]
        #print (fleet_capacity)
    if (PV_B2B.D_truckload[i] < fleet_capacity):
        fleet_capacity = fleet_capacity - PV_B2B.D_truckload[i]
        #print (fleet_capacity)
    else: 
        PV_B2B.mode_choice[i] = "For-hire Truck"
    # print (i)    

FH_B2B=pd.concat([FH_B2B,PV_B2B[PV_B2B['mode_choice']=="For-hire Truck"]])

In [82]:
## Get shipper's shipment the entire truckload (sum by seller ID ) for each day: 
##   Need to find a couple of carriers (like making a contract with carriers)
##### Update requried: This could be updated later with contract-related modeling 
FH_Seller= FH_B2B.groupby(['SellerID', 'SellerZone'])['D_truckload'].sum()
FH_Seller=FH_Seller.to_frame()
FH_Seller.reset_index(level=(0,1), inplace=True)

In [89]:
#FH_Seller.head()

In [90]:
### Get trucking (for-hire trucking) and postals seperately and assigned capacity in pounds
##### Update requried: Need to check the capacity carefully!!! 
#firms_sub=firms[['BusID','md_veh','hd_veh']]
#warehouses = warehouses.merge(firms_sub, on='BusID', how='left')
warehouses['capacity']=(warehouses.md_veh*14000+warehouses.hd_veh*40000)*1.05
truckings=warehouses.query('Industry_NAICS6_Make=="484000"')
#postals=warehouses.query('Industry_NAICS6_Make=="492000"')

In [91]:
###  Calculate distance between two meso zone: currenlty Euclidian dist
#####  Update requried: Need to get network distance? -> please update "dist_df=pd.read_csv(...)""  
def dist_cal(org_meso, dest_meso):
    dist = dist_df[(dist_df['Origin']==org_meso) & (dist_df['Destination']==dest_meso)].dist.values[0]
    if (dist == 0):
        dist =random.uniform(1,10)
    return dist

In [92]:
### new version
### Assign the carrier s.t.: 
### 1. distance with capacity 
### 2. select a carrier from the candidate list with distance weighted probability 
def carrier_sel(SellerZone, D_truckload):
    sel_dist_df = dist_df[(dist_df.Origin==SellerZone) & (dist_df.dist<50)]
    candidate_busid= truckings[(truckings['MESOZONE'].isin(sel_dist_df.Destination.unique())) & (truckings['capacity'] >=D_truckload)][['BusID','MESOZONE']] 
    
    if (candidate_busid.shape[0]  < 10):
        sel_dist_df = dist_df[(dist_df.Origin==SellerZone) & (dist_df.dist<100)]
        candidate_busid= truckings[(truckings['MESOZONE'].isin(sel_dist_df.Destination.unique())) & (truckings['capacity'] >=D_truckload)][['BusID','MESOZONE']] 
    if (candidate_busid.shape[0]  < 10):
        sel_dist_df = dist_df[(dist_df.Origin==SellerZone) & (dist_df.dist<200)]
        candidate_busid= truckings[(truckings['MESOZONE'].isin(sel_dist_df.Destination.unique())) & (truckings['capacity'] >=D_truckload)][['BusID','MESOZONE']] 


    candidate_busid=candidate_busid.sample(10)
    
    candidate_busid["org_CBG"]=SellerZone
    candidate_busid= candidate_busid.reset_index(drop=True)

    candidate_busid['ttime']=candidate_busid.apply(lambda x: dist_cal(x['org_CBG'], x['MESOZONE']), axis=1)
    candidate_busid['inv_sqr_tt']=candidate_busid.apply(lambda x: 1/x['ttime'], axis=1)
    sum_pro=np.sum(candidate_busid['inv_sqr_tt'])
    candidate_busid['prob']=candidate_busid.apply(lambda x: x['inv_sqr_tt']/sum_pro, axis=1)
    candidate_busid['cum_prob_ub']=candidate_busid['prob'].cumsum()
    candidate_busid['cum_prob_lb']=np.nan_to_num(candidate_busid['cum_prob_ub'].shift())
    candidate_busid['cum_prob_ub'].iloc[-1] = 1.01
    r_num = random.uniform(0,1)
    sel_busID=candidate_busid[(candidate_busid['cum_prob_ub'] > r_num) & (candidate_busid['cum_prob_lb'] <= r_num)].BusID.values[0]
    truckings.loc[(truckings.BusID == sel_busID),"capacity"] = truckings.loc[(truckings.BusID == sel_busID),"capacity"] - D_truckload
    return sel_busID

In [93]:
## Let check the size
FH_Seller.shape

(1216, 3)

In [94]:
### Run the carrier matching for for-hire truck
FH_Seller['assigned_carrier']=0

for i in range(0,FH_Seller.shape[0]):
    FH_Seller['assigned_carrier'][i]=carrier_sel(FH_Seller['SellerZone'][i], FH_Seller['D_truckload'][i])
    print (i)

/Users/kjeong/anaconda3/envs/python38/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/kjeong/anaconda3/envs/python38/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/var/folders/kr/2j58542j2_7chf9_h8bmr30mb0_4n0/T/ipykernel_42823/3992304902.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [97]:
## temporary saving: For-hire Seller with assigned_carrier
FH_Seller.to_csv ('../Sim_outputs/temp_save/FH_Seller.csv', index = False, header=True)

## read tempoary data
#FH_Seller = pd.read_csv('../Sim_outputs/temp_save/FH_Seller.csv', header=0, sep=',')

In [98]:
## Assigned the carrier ID to each shipment pair 
B2BF_FH=FH_B2B.merge(FH_Seller[['SellerID', 'assigned_carrier' ]], on='SellerID', how='left')

In [99]:
B2BF_FH = B2BF_FH.dropna(subset=['assigned_carrier'])

In [100]:
B2BF_PV = PV_B2B

In [101]:
B2BF_PV = B2BF_PV.sort_values(by=['SellerID', 'SCTG_Group'])
B2BF_FH = B2BF_FH.sort_values(by=['SellerID', 'SCTG_Group'])
B2BF_PV =B2BF_PV.reset_index(drop=True)
B2BF_FH =B2BF_FH.reset_index(drop=True) 

In [102]:
### Create Payload file 

In [103]:
payloads = pd.DataFrame(columns = ['payload_id', 
'carrier_id',
'sequence_id',
'tour_id',
'commodity',
'weight',
'job',
'pu_zone',
'del_zone',
'pu_stop_duration',
'del_stop_duration',
'pu_tw_lower',
'pu_tw_upper',
'del_tw_lower',
'del_tw_upper',
'pu_arrival_time',
'del_arrival_time'])

In [104]:
payloads['payload_id']= np.arange(B2BF_PV.shape[0])
payloads['carrier_id']= B2BF_PV['SellerID']
#payloads['sequence_id']=
#payloads['tour_id']=
payloads['commodity']=B2BF_PV['SCTG_Group']
payloads['weight']=B2BF_PV['D_truckload']
payloads['job']= 'delivery'
#payloads['pu_zone']=
payloads['del_zone']=B2BF_PV['BuyerZone']
#payloads['pu_stop_duration']=
## need to fix this distribution
load_min =B2BF_PV.D_truckload.min()
load_max = B2BF_PV.D_truckload.max()
payloads['del_stop_duration']=B2BF_PV.apply(lambda x: int(np.random.gamma(2, 1, 1)[0]*((x['D_truckload'] -load_min)/(load_max-load_min))*120), axis=1)
payloads['del_stop_duration'] =payloads['del_stop_duration'].apply(lambda x: random.randint(5,20) if x <5 else x)  
payloads['del_stop_duration']=payloads['del_stop_duration'].apply(lambda x: 90 if x >90 else x)
#payloads['pu_tw_lower']=
#payloads['pu_tw_upper']=
payloads['del_tw_lower']=60*5
payloads['del_tw_upper']=60*20
#payloads['pu_arrival_time']=
#payloads['del_arrival_time']=  

In [105]:
payloads_FH = pd.DataFrame(columns = ['payload_id', 
'carrier_id',
'sequence_id',
'tour_id',
'commodity',
'weight',
'job',
'pu_zone',
'del_zone',
'pu_stop_duration',
'del_stop_duration',
'pu_tw_lower',
'pu_tw_upper',
'del_tw_lower',
'del_tw_upper',
'pu_arrival_time',
'del_arrival_time'])

In [106]:
payloads_FH['payload_id']= np.arange(B2BF_PV.shape[0],B2BF_FH.shape[0])
payloads_FH['carrier_id']= B2BF_FH['assigned_carrier']
#payloads['sequence_id']=
#payloads['tour_id']=
payloads_FH['commodity']=B2BF_FH['SCTG_Group']
payloads_FH['weight']=B2BF_FH['D_truckload']
payloads_FH['job']= 'pickup_delivery'
payloads_FH['pu_zone']=B2BF_FH['SellerZone']
payloads_FH['del_zone']=B2BF_FH['BuyerZone']
load_min =B2BF_FH.D_truckload.min()
load_max = B2BF_FH.D_truckload.max()
# need to put max 
payloads_FH['pu_stop_duration']= B2BF_FH.apply(lambda x: int(np.random.gamma(2, 1, 1)[0]*((x['D_truckload'] -load_min)/(load_max-load_min))*120), axis=1)
payloads_FH['pu_stop_duration'] =payloads_FH['pu_stop_duration'].apply(lambda x: random.randint(5,20) if x <5 else x)   
payloads['pu_stop_duration']=payloads['pu_stop_duration'].apply(lambda x: 90 if x >90 else x)
## need to fix this distribution
payloads_FH['del_stop_duration']=B2BF_FH.apply(lambda x: int(np.random.gamma(2, 1, 1)[0]*((x['D_truckload'] -load_min)/(load_max-load_min))*120), axis=1)
payloads_FH['del_stop_duration'] =payloads_FH['del_stop_duration'].apply(lambda x: random.randint(5,20) if x <5 else x)                                           
payloads['del_stop_duration']=payloads['del_stop_duration'].apply(lambda x: 90 if x >90 else x)
payloads_FH['pu_tw_lower']=60*5
payloads_FH['pu_tw_upper']=60*20
payloads_FH['del_tw_lower']=60*5
payloads_FH['del_tw_upper']=60*20
#payloads['pu_arrival_time']=
#payloads['del_arrival_time']=  

In [107]:
payloads=pd.concat([payloads, payloads_FH])

In [108]:
payloads.to_csv (fdir_main_output+fnm_payload, index = False, header=True)

In [109]:
### End Create Payload file 

In [110]:
### departure time distribution
CBGzone_df.GEOID=CBGzone_df.GEOID. astype(str). astype(int)
df_dpt_dist=df_dpt_dist.merge(CBGzone_df[["GEOID",'MESOZONE']], left_on="cbg_id", right_on="GEOID", how='left')

In [111]:
def depot_time_depart(zone_id):
    try:
        df_temp = df_dpt_dist[df_dpt_dist['MESOZONE']==zone_id].reset_index()
        df_temp['cdf_low']=0.0
        df_temp['cdf_up']=0.0
        for i in range(1,df_temp.shape[0]):
            df_temp.loc[i,'cdf_up']=df_temp.loc[i-1,'cdf_up']+df_temp.loc[i,'Trip_pdf']
            df_temp.loc[i,'cdf_low']=df_temp.loc[i-1,'cdf_up']
        pro_time= random.uniform(0, 0.9999999)    
        d_time=df_temp[(df_temp['cdf_low']<= pro_time) & (df_temp['cdf_up'] >  pro_time)]['start_hour'].values[0]
        d_time=random.randrange(d_time*60, (d_time+1)*60, 10)
    except:
        d_time= random.randrange(7*60, 17*60, 10)
    return d_time    

In [112]:
def depot_time_close(d_time):
    c_time = d_time+3
    if c_time >=24: 
        c_time = 24*60
    elif c_time >= 17 and c_time <24:
        c_time= random.randrange(c_time*60, 24*60, 30)
    else:     
        c_time=random.randrange(17*60, 22*60, 30)
    return c_time

In [113]:
### Create Carrier file

In [114]:
PV_T_D =B2BF_PV.groupby(['SellerID', 'SellerZone'])['D_truckload'].sum()
PV_T_D=PV_T_D.to_frame()
PV_T_D.reset_index(level=(0,1), inplace=True)
firms_sub=firms[['BusID','MESOZONE','md_veh','hd_veh']]
firms_sub.rename({'BusID':'SellerID'},axis=1, inplace=True)
PV_T_D = PV_T_D.merge(firms_sub, on='SellerID', how='left')

/Users/kjeong/anaconda3/envs/python38/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [115]:
carriers = pd.DataFrame(columns = ['carrier_id',
'firm_id',
'depot_zone',
'contract_firms',
'num_veh_type_1',
'num_veh_type_2',
'num_veh_type_3',
'num_veh_type_4',
'num_veh_type_5',
'num_veh_type_6',
'num_veh_type_7',
'num_veh_type_8',
'num_veh_type_9',
'depot_lower',
'depot_upper',
'depot_time_before',
'depot_time_after'])

In [116]:
carriers['carrier_id']=PV_T_D['SellerID']
carriers['firm_id']=PV_T_D['SellerID']
carriers['depot_zone']=PV_T_D['SellerZone']
carriers['contract_firms']=PV_T_D['SellerID'].apply(lambda x: [x])
carriers['num_veh_type_1']=PV_T_D['md_veh']
carriers['num_veh_type_2']=PV_T_D['hd_veh']
carriers['depot_lower']= carriers['depot_zone'].apply(depot_time_depart)
carriers['depot_upper']= carriers['depot_lower'].apply(depot_time_close)
carriers['depot_time_before']= [random.randrange(0,30, 5) for j in carriers.index]
carriers['depot_time_after']= [random.randrange(0,30, 5) for j in carriers.index]

In [117]:
temp_FH_T_D = B2BF_FH.groupby('assigned_carrier')['SellerID'].apply(list).reset_index(name='contract_firms')

In [118]:
temp_FH_T_D.rename({'assigned_carrier': 'SellerID'},axis=1, inplace=True)

In [119]:
temp_FH_T_D = temp_FH_T_D.merge(firms_sub, on='SellerID', how='left')

In [120]:
temp = pd.DataFrame(columns = ['carrier_id',
'firm_id',
'depot_zone',
'contract_firms',
'num_veh_type_1',
'num_veh_type_2',
'num_veh_type_3',
'num_veh_type_4',
'num_veh_type_5',
'num_veh_type_6',
'num_veh_type_7',
'num_veh_type_8',
'num_veh_type_9',
'depot_lower',
'depot_upper',
'depot_time_before',
'depot_time_after'])
temp['carrier_id']=temp_FH_T_D['SellerID']
temp['firm_id']=temp_FH_T_D['SellerID']
temp['depot_zone']=temp_FH_T_D['MESOZONE']
temp['contract_firms']=temp_FH_T_D['contract_firms']
temp['num_veh_type_1']=temp_FH_T_D['md_veh']
temp['num_veh_type_2']=temp_FH_T_D['hd_veh']
temp['depot_lower']= temp['depot_zone'].apply(depot_time_depart)
temp['depot_upper']= temp['depot_lower'].apply(depot_time_close)
temp['depot_time_before']= [random.randrange(5,40, 5) for j in temp.index]
temp['depot_time_after']= [random.randrange(5,40, 5) for j in temp.index]

In [121]:
carriers=pd.concat([carriers, temp])

In [122]:
carriers.to_csv (fdir_main_output+fnm_carrier, index = False, header=True)

In [123]:
### End Create Carrier file

In [124]:
### Create Vehicle type
##### Update requried: more vehicle types?

In [125]:
vehicle_types= pd.DataFrame(columns = ['veh_type_id',
'veh_category',
'veh_class',
'body_type',
'commodities',
'weight', 
'length',
'payload_capacity_weight',
'payload_capacity_cbf',
'max_speed',
'primary_fuel_type',
'secondary_fuel_type',
'primary_fuel_rate',
'secondary_fuel_rate',
'Automation level',
'monetary cost'])

In [126]:
MD= pd.DataFrame(data={'veh_type_id': [1],
'veh_category': ['MD'],
'veh_class':['NA'],
'body_type':['NA'],
'commodities':[[1,2,3,4]],
'weight':[20000], 
'length':['NA'],
'payload_capacity_weight':[14000],
'payload_capacity_cbf':['NA'],
'max_speed':['NA'],
'primary_fuel_type':['NA'],
'secondary_fuel_type':['NA'],
'primary_fuel_rate':['NA'],
'secondary_fuel_rate':['NA'],
'Automation level':['NA'],
'monetary cost':['NA']})   

HD= pd.DataFrame(data={'veh_type_id': [2],
'veh_category': ['HD'],
'veh_class':['NA'],
'body_type':['NA'],
'commodities':[[1,2,3,4]],
'weight':[60000], 
'length':['NA'],
'payload_capacity_weight':[40000],
'payload_capacity_cbf':['NA'],
'max_speed':['NA'],
'primary_fuel_type':['NA'],
'secondary_fuel_type':['NA'],
'primary_fuel_rate':['NA'],
'secondary_fuel_rate':['NA'],
'Automation level':['NA'],
'monetary cost':['NA']})
                 
vehicle_types=vehicle_types.append(MD)
vehicle_types=vehicle_types.append(HD)

In [127]:
vehicle_types.to_csv (fdir_main_output+fnm_vtype, index = False, header=True)

In [ ]:
### End Create Vehicle type